In [1]:
import polars as pl

import rustystats as rs

data = pl.read_parquet("https://raw.githubusercontent.com/PricingFrontier/pricing-data-example/917c853e256df8d5814721ab56f72889a908bb08/data/processed/frequency_set.parquet")

train_data = data.filter(pl.col("Group") <= "3")
test_data = data.filter(pl.col("Group") == "4")
holdout_data = data.filter(pl.col("Group") == "5")

In [2]:
#creates analysis/exploration.json
exploration = rs.explore_data(
    data=train_data,
    response="ClaimCount",
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ],
    exposure="Exposure",
)

In [3]:
formula = """
    ClaimCount ~ 
    ms(VehAge, df=4, increasing=True)
"""

result = rs.glm(
    formula,
    train_data, 
    family="poisson", 
    offset="Exposure").fit(
)

print(result.summary())

                                 GLM Results                                  

Family:              Poisson         No. Observations:        406439
Link Function:       (default)       Df Residuals:            406434
Method:              IRLS            Df Model:                     4
Scale:               1.0000          Iterations:                  25

Log-Likelihood:         -117115.3232 Deviance:                192682.9966
AIC:                     234240.6463 Null Deviance:           132328.3090
BIC:                     234295.2223 Pearson chi2:              385012.02
Converged:           False          

------------------------------------------------------------------------------
Variable                 Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept             -1.1961     0.0181  -65.909  <0.0001   [ -1.2317,  -1.1605]  ***
ms(VehAge, 1/4, +)     0.0000     0.0202    0.000  

In [4]:
#writes analysis/diagnostics
result.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': '\n    ClaimCount ~ \n    ms(VehAge, df=4, increasing=True)\n', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 5, 'df_resid': 406434, 'converged': False, 'iterations': 25}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=76961.22, loss=0.474076, deviance=192683.0, log_likelihood=-117115.32, aic=234240.65, gini=-0.1234, auc=0.4383, ae_ratio=0.2782, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1559.24, actual=0.82412, predicted=0.318624, ae_ratio=2.5865), DecileMetrics(decile=2, n=40643, exposure=3527.75, actual=0.270711, predicted=0.326543, ae_ratio=0.829), DecileMetrics(decile=3, n=40643, exposure=7098.42, actual=0.211315, predicted=0.323718, ae_ratio=0.6528), DecileMetrics(decile=4, n=40643, exposure=11646.21, actual=0.146829, predicted=0.325946, ae_ratio=0.4505), DecileMetrics(decile=5, n=40643, exposure=17577.38

In [5]:
result.predict(train_data)

array([0.03094064, 0.34222329, 0.0200687 , ..., 0.01368893, 0.3853874 ,
       0.35045853], shape=(406439,))

In [8]:
result2 = rs.glm_dict(
    response="ClaimCount",
    terms={
        "Region": {"type": "target_encoding"},
        "BonusMalus": {"type": "linear"},
        "BonusMalus2": {"type": "expression", "expr": "BonusMalus ** 2"},
        "VehAge": {"type": "ms", "df": 4, "monotonicity": "increasing"},
        "DrivAge": {"type": "linear", "monotonicity": "increasing"},
        "DrivAge2": {"type": "expression", "expr": "DrivAge ** 2", "monotonicity": "increasing"},
        "VehBrand": {"type": "target_encoding"},
        "Area": {"type": "target_encoding"},
    },
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit()

In [9]:
#writes analysis/diagnostics
result2.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ TE(Region) + BonusMalus + I(BonusMalus ** 2) + DrivAge + I(DrivAge ** 2) + TE(VehBrand) + TE(Area)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 12, 'df_resid': 406427, 'converged': False, 'iterations': 25}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=76090.92, loss=0.463546, deviance=188403.13, log_likelihood=-114975.39, aic=229974.78, gini=0.1515, auc=0.5758, ae_ratio=0.2813, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1709.95, actual=0.615223, predicted=0.266432, ae_ratio=2.3091), DecileMetrics(decile=2, n=40643, exposure=3732.61, actual=0.302737, predicted=0.302428, ae_ratio=1.001), DecileMetrics(decile=3, n=40643, exposure=7575.46, actual=0.185203, predicted=0.290317, ae_ratio=0.6379), DecileMetrics(decile=4, n=40643, exposure=12721.7, actual=0.134888, predicted=0.288663, ae_ratio=0.4673)